# Metadata collection with ZnTrack

ZnTrack allows for the collection of some metadata.
One example is measuring the execution time of Nodes or even methods inside the Nodes easily.
This can be achieved by using the `@TimeIt` decorator which is shown in the following example.

In [1]:
from zntrack import Node, zn, config
from zntrack.metadata import TimeIt
from time import sleep

config.nb_name = "05_metadata.ipynb"

In [2]:
from zntrack.utils import cwd_temp_dir

temp_dir = cwd_temp_dir()

In [3]:
!git init
!dvc init

Initialized empty Git repository in C:/Users/fabia/AppData/Local/Temp/tmp67vxo_51/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [4]:
class SleepNode(Node):
    metadata = zn.metadata()

    @TimeIt
    def run(self):
        self.sleep_1s()
        self.sleep_2s()

    @TimeIt
    def sleep_1s(self):
        sleep(1)

    def sleep_2s(self):
        sleep(2)

In [5]:
SleepNode().write_graph(no_exec=False)

2022-01-13 21:14:07,252 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-01-13 21:14:07,253 (WARNING): Converting 05_metadata.ipynb to file SleepNode.py
2022-01-13 21:14:09,787 (WARNING): --- Writing new DVC file! ---
2022-01-13 21:14:09,788 (WARNING): You will not be able to see the stdout/stderr of the process in real time!
2022-01-13 21:14:15,249 (INFO): Running stage 'SleepNode':
> python -c "from src.SleepNode import SleepNode; SleepNode.load(name='SleepNode').run_and_save()" 
Creating 'dvc.yaml'
Adding stage 'SleepNode' in 'dvc.yaml'
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml



In [6]:
!dvc metrics show

Path                                   metadata.run:timeit    metadata.sleep_1s:timeit
nodes\SleepNode\metrics_no_cache.json  3.03                   1.015


We can also time a sinlge function multiple times, using the following example:

In [7]:
class SleepNodeMulti(Node):
    metadata = zn.metadata()

    @TimeIt
    def run(self):
        self.sleep(1)
        self.sleep(2)

    @TimeIt
    def sleep(self, time):
        sleep(time)

In [8]:
SleepNodeMulti().write_graph(no_exec=False)

2022-01-13 21:14:16,917 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-01-13 21:14:16,918 (WARNING): Converting 05_metadata.ipynb to file SleepNodeMulti.py
2022-01-13 21:14:19,191 (WARNING): --- Writing new DVC file! ---
2022-01-13 21:14:19,191 (WARNING): You will not be able to see the stdout/stderr of the process in real time!
2022-01-13 21:14:24,447 (INFO): Running stage 'SleepNodeMulti':
> python -c "from src.SleepNodeMulti import SleepNodeMulti; SleepNodeMulti.load(name='SleepNodeMulti').run_and_save()" 
Adding stage 'SleepNodeMulti' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml



In [9]:
!dvc metrics show

Path                                        metadata.run:timeit    metadata.sleep:timeit    metadata.sleep_1:timeit    metadata.sleep_1s:timeit
nodes\SleepNode\metrics_no_cache.json       3.03                   -                        -                          1.015
nodes\SleepNodeMulti\metrics_no_cache.json  3.025                  1.01                     2.015                      -


One can also access the metrics directly within Python. This is possible, because they are just another `zn.metrics` which is automatically added when using one of the given metadata decorators.

In [10]:
SleepNodeMulti.load().metadata

{'sleep:timeit': 1.0100021362304688,
 'sleep_1:timeit': 2.014997959136963,
 'run:timeit': 3.0250000953674316}

In [ ]:
temp_dir.cleanup()